In [3]:
import pandas as pd
import re


In [4]:
df = pd.read_csv('jumia_enhanced_data_20250926_195337.csv')  # Replace with your file name
print("Initial data shape:", df.shape)
df.head()

Initial data shape: (4946, 16)


,product_id,product_name,brand,model,category,category_key,current_price,original_price,discount_percent,price_tier,value_score,is_on_sale,url,scraped_date,scraped_time,scraped_timestamp
0,JUM_0001,NaN,NaN,NaN,Laptops & Computers,ordinateurs-pc,2779.0,NaN,0.0,Budget,60.0,False,https://www.jumia.ma/lenovo-notebook-thinkpad-...,2025-09-26,19:02:54,2025-09-26T19:02:54.972612
1,JUM_0002,NaN,NaN,NaN,Laptops & Computers,ordinateurs-pc,2590.0,NaN,0.0,Budget,60.0,False,https://www.jumia.ma/hp-elitebook-840-g5-intel...,2025-09-26,19:02:55,2025-09-26T19:02:55.277587
2,JUM_0003,NaN,NaN,NaN,Laptops & Computers,ordinateurs-pc,3190.0,NaN,0.0,Mid-range,50.0,False,https://www.jumia.ma/hp-elitebook-840-g5-core-...,2025-09-26,19:02:55,2025-09-26T19:02:55.490303
3,JUM_0004,NaN,NaN,NaN,Laptops & Computers,ordinateurs-pc,3799.0,NaN,0.0,Mid-range,50.0,False,https://www.jumia.ma/hp-pc-portable-elitebook-...,2025-09-26,19:02:55,2025-09-26T19:02:55.595837
4,JUM_0005,NaN,NaN,NaN,Laptops & Computers,ordinateurs-pc,2589.0,NaN,0.0,Budget,60.0,False,https://www.jumia.ma/hp-pc-portable-hp-elitebo...,2025-09-26,19:02:55,2025-09-26T19:02:55.683804


In [5]:
df = df.replace("N/A", pd.NA)
df['brand'] = df['brand'].fillna('Unknown')
df['model'] = df['model'].fillna('Unknown')
df['product_name'] = df['product_name'].fillna('Unknown')


In [6]:
numeric_cols = ['current_price', 'original_price', 'discount_percent', 'value_score']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [7]:
df['calculated_discount'] = ((df['original_price'] - df['current_price']) / df['original_price'] * 100).round(2)
df['discount_percent'] = df.apply(
    lambda x: x['calculated_discount'] if pd.isna(x['discount_percent']) or x['discount_percent'] == 0 else x['discount_percent'], axis=1
)


In [8]:
df['scraped_datetime'] = pd.to_datetime(df['scraped_timestamp'])

In [9]:
df['product_name'] = df['product_name'].str.strip().str.title()

In [10]:
df['is_on_sale'] = df['is_on_sale'].astype(bool)

In [11]:
def categorize_price(price):
    if price < 1500:
        return 'Budget'
    elif price < 4000:
        return 'Mid-range'
    else:
        return 'Premium'

df['price_tier'] = df['current_price'].apply(categorize_price)

In [12]:
def extract_ram(name):
    match = re.search(r'(\d+)\s?Go', str(name))
    return int(match.group(1)) if match else pd.NA

def extract_ssd(name):
    match = re.search(r'(\d+)\s?(Go\s?SSD|SSD)', str(name), re.IGNORECASE)
    return int(match.group(1)) if match else pd.NA

df['RAM_GB'] = df['product_name'].apply(extract_ram)
df['SSD_GB'] = df['product_name'].apply(extract_ssd)

In [13]:
df.to_csv('jumia_cleaned.csv', index=False)
print("Cleaned data saved. Shape:", df.shape)

Cleaned data saved. Shape: (4946, 20)


In [14]:
# Convert to title case for consistency
df['brand'] = df['brand'].str.strip().str.title()

In [15]:
# Define common replacements for standardization
brand_replacements = {
    'Samsung': 'Samsung',
    'Hp': 'HP',
    'Dell': 'Dell',
    'Lenovo': 'Lenovo',
    'Asus': 'ASUS',
    'Xiaomi': 'Xiaomi',
    'Apple': 'Apple',
    'Iphone': 'Apple',  # Group iPhone listings under Apple
    'Lg': 'LG',
    'Tcl': 'TCL',
    'Hisense': 'Hisense',
    'Sony': 'Sony',
    'Playstation': 'PlayStation',
    'Xbox': 'Xbox',
    'Nokia': 'Nokia',
    'Huawei': 'Huawei'
}


In [16]:
# Apply the standardization
def standardize_brand(brand):
    # Check for direct matches (case-insensitive search for flexibility)
    for key, value in brand_replacements.items():
        if key.lower() in str(brand).lower():
            return value
    return brand

df['brand'] = df['brand'].apply(standardize_brand)

print(f"Unique Brands After Standardization: {df['brand'].nunique()}")

Unique Brands After Standardization: 489


In [20]:
# Recalculate and enforce logic: if current_price is missing or zero, assume original_price is zero too.
df.loc[df['current_price'] <= 0, ['original_price', 'discount_percent', 'calculated_discount']] = 0

In [25]:
condition = df['current_price'] >= df['original_price']

In [26]:
# Use the boolean condition to set each column individually
df.loc[condition, 'original_price'] = df['current_price']
df.loc[condition, 'discount_percent'] = 0.0
df.loc[condition, 'calculated_discount'] = 0.0